In [70]:
import pandas as pd
import os
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [71]:
load_dotenv()

True

In [72]:
class RatingPrediction(BaseModel):
    predicted_stars: Literal[1, 2, 3, 4, 5] = Field(
        description="Predicted Yelp star rating from 1 to 5"
    )
    explanation: str = Field(
        description="One-sentence justification for the predicted rating, (50 words)"
    )


In [74]:
llm = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct",
               api_key=os.environ["GROQ_API_KEY"])

In [ ]:
structured_llm = llm.with_structured_output(RatingPrediction)

In [76]:
prompt_zero = PromptTemplate(
    input_variables=["review_text"],
    template="""
You are an expert sentiment classifier for Yelp reviews.

Analyze the review and predict the appropriate star rating (1–5).
Base your decision on overall sentiment, tone, and complaints or praise.

Hard constraint:
- The ENTIRE response must be 100 words or fewer.

Review:
"{review_text}"
"""
)


In [77]:
chain_zero = prompt_zero | structured_llm

In [78]:
prompt_few = PromptTemplate(
    input_variables=["review_text"],
    template="""
You are an expert Yelp review classifier.

Examples:

Review: "The food was cold and the waiter was rude."
Rating: 1 star
Reason: Strong negative sentiment toward food and service.

Review: "It was fine, nothing memorable."
Rating: 3 stars
Reason: Neutral experience without strong positives or negatives.

Review: "Amazing food and wonderful staff!"
Rating: 5 stars
Reason: Highly positive sentiment and strong praise.

Now classify the following review:

Hard constraint:
- The ENTIRE response must be 100 words or fewer.

Review:
"{review_text}"
"""
)


In [79]:
chain_few = prompt_few | structured_llm

In [80]:
prompt_structured = PromptTemplate(
    input_variables=["review_text"],
    template="""
You are a Yelp review analyst.

Step 1: Determine the overall sentiment (negative, neutral, positive).
Step 2: Map that sentiment to a star rating from 1 to 5.
Step 3: Provide a brief justification.

Hard constraint:
- The ENTIRE response must be 100 words or fewer.

Review:
"{review_text}"
"""
)


In [81]:
chain_structured = prompt_structured | structured_llm


In [82]:
df = pd.read_csv(r"D:\Desktop\fynd\data\yelp.csv")



In [83]:
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [84]:
df = df[df["text"].str.split().str.len() <=25]
df.shape

(936, 10)

In [85]:
df = df[["stars", "text"]]
df.head()

,stars,text
2,4,love the gyro plate. Rice is so good and I als...
21,5,This place shouldn't even be reviewed - becaus...
42,4,"Solid food, great device, and casual environme..."
43,4,"my son really enjoys islands, great food, casu..."
68,4,The best sweet and sour soup ever! Food is de...


In [86]:
df = df.sample(200, random_state=42)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 3449 to 9587
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stars   200 non-null    int64 
 1   text    200 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


In [87]:
df.drop_duplicates(inplace=True)
df.shape

(200, 2)

In [88]:
from tqdm.auto import tqdm
from pydantic import ValidationError

def run_predictions(chain, df):
    results = []
    for i, (_, row) in enumerate(tqdm(df.iterrows(),
                                      total=len(df),
                                      desc="Running predictions"), start=1):
        try:
            output = chain.invoke({"review_text": str(row["text"])})
            pred = output.predicted_stars
            expl = output.explanation
        except ValidationError as e:
            # fallback when model returns bad / non-JSON output
            pred = 3
            expl = f"Parse error: {e.errors()[0]['type']}"

        results.append({
            "true_stars": row["stars"],
            "predicted_stars": pred,
            "explanation": expl,
        })
    return pd.DataFrame(results)

        


In [ ]:
pred_zero = run_predictions(chain_zero, df)

Running predictions: 100%|██████████| 200/200 [29:08<00:00,  8.74s/it]


In [94]:
pred_few = run_predictions(chain_few, df)

Running predictions: 100%|██████████| 200/200 [31:15<00:00,  9.38s/it]


In [97]:
pred_structured = run_predictions(chain_structured, df)

Running predictions: 100%|██████████| 200/200 [29:25<00:00,  8.83s/it]


In [98]:
pred_zero.to_csv("predictions_zero_shot.csv", index=False)
pred_few.to_csv("predictions_few_shot.csv", index=False)
pred_structured.to_csv("predictions_structured.csv", index=False)


In [99]:
def accuracy(pred_df):
    return (pred_df["true_stars"] == pred_df["predicted_stars"]).mean()



In [100]:
acc_zero = accuracy(pred_zero)
acc_few = accuracy(pred_few)
acc_structured = accuracy(pred_structured)

acc_zero, acc_few, acc_structured


(np.float64(0.585), np.float64(0.615), np.float64(0.6))